In [10]:
import requests
def get_current_price(symbol):
    url = f'https://api.binance.com/api/v3/ticker/price'
    params = {'symbol': symbol+"USDT"}
    response = requests.get(url, params=params)
    data = response.json()
    usdt_price = float(data['price'])
    return usdt_price
coin_names = {
  'bitcoin': 'BTC',
  'ethereum': 'ETH',
  'binance coin': 'BNB',
  'ripple': 'XRP',
  'solana': 'SOL',
  'cardano': 'ADA',
  'dogecoin': 'DOGE',
  'polygon': 'MATIC',
  'polkadot':"DOT",
  'avalanche': 'AVAX'
}
def get_current_price_wrapper(input):
  if input['coin_name'].lower() in coin_names:
    return f"""\n ######### Context ############
The price of {input} now is : {str(get_current_price(coin_names[input['coin_name'].lower()]))}
###########################"""
  else:
     return f" The price of {input} now is UNKNOWN. "

In [ ]:
from openai import OpenAI
import openai
from pydantic import BaseModel
import json

client = OpenAI(api_key = "sk-sJAILfYY4hF8aVTM73A26fB09c834c7b8c41D4CeB652Fe95",base_url="https://openai.ss-gpt.com/v1")

class coin(BaseModel):
  coin_name:str

q1 = "What is the price of bitcoin now?"
q2 = "What is bitcoin?"
question = "What is the price of PiggyCoin 2 years ago? Do not give me the price now."

def query_with_tools(input):
  tools = [openai.pydantic_function_tool(coin)]
  print("query_with_tools: ", input)
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "assistant", "content": """You are an Web3 assistant who can answer according to context and call tools to get context. Please call tools only if the context is missing and you need to retrieve the required information to answer questions. Otherwise output the answer directly."""},
      {"role": "user", "content": input}
    ],
    tools=tools
    #response_format={ "type": "json_object" },
  )
  print("Completion: ",completion)
  return completion
completion = query_with_tools(question)
print(completion)
answer = completion.choices[0].message.content
if completion.choices[0].message.tool_calls!=None:
  function_name = completion.choices[0].message.tool_calls[0].function.name
  function_args = completion.choices[0].message.tool_calls[0].function.arguments

  print("The argument: ")
  print(completion.choices[0].message.tool_calls[0].function.arguments) 
  data = json.loads(function_args)
  if data:
    if (function_name=="coin"):
      tool_output = get_current_price_wrapper(data)
    try:
      answer = query_with_tools(q1 +"\n"+ tool_output).choices[0].message.content
    except:
      pass
print("Question: ",question)
print("Answer: : ",answer)
     
# completion


QUERY:  What is the price of PiggyCoin 2 years ago? Do not give me the price now.
Completion:  ChatCompletion(id='chatcmpl-AQnwjDH2D9wiqCSP6Sx7RCiArYgTH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_icEiCta4EeufiDPWAt4W0GCP', function=Function(arguments='{"coin_name":"PiggyCoin"}', name='coin'), type='function')]))], created=1730951885, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_d54531d9eb', usage=CompletionUsage(completion_tokens=16, prompt_tokens=115, total_tokens=131, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletion(id='chatcmpl-AQnwjDH2D9wiqCSP6Sx7RCiArYgTH', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=Ch